In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2
import psycopg2.extras

In [5]:
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "localhost"
DB_NAME = "postgres"

CONNECTION_URL = f"postgres://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

In [7]:
# establish connection to postgres
conn = psycopg2.connect(CONNECTION_URL)
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT 'hello world'")
print(cursor.fetchone())

('hello world',)
